Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from sklearn.linear_model import LogisticRegression

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (10000, 28, 28) (10000,)
Validation set (2500, 28, 28) (2500,)
Test set (2500, 28, 28) (2500,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (10000, 784) (10000, 10)
Validation set (2500, 784) (2500, 10)
Test set (2500, 784) (2500, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [57]:
# Code taken from 2_fullyconnected and modified by LDP
# Introduce L2 regularization for Logistic Regression

batch_size = 128
# Range of values for alpha parameter (regularization parameter)
alpha_vals = np.logspace(0, -4, 20)
accuracy_vals = []

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_alpha_reg = tf.placeholder(tf.float32)

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  
  # Compute the loss with L2
  # L2 is simply the sum of the squares of the coefs, without bias
  # Alpha is the regularization parameter
  # Choosing alpha lower -> better coefs
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) + \
    tf_alpha_reg * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [58]:
num_steps = 3001
for alpha_reg in alpha_vals:
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        # Need to reshape, because the new PlaceHolder has different dimensions
        batch_labels = batch_labels.reshape(batch_size,10)
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_alpha_reg : alpha_reg}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
#         if ((step % 500) == 0):
#           print("Minibatch loss at step %d: %f" % (step, l))
#           print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#           print("Validation accuracy: %.1f%%" % accuracy(
#             valid_prediction.eval(), valid_labels))

      accuracy_value = accuracy(test_prediction.eval(), test_labels)
      print("Alpha val:%.5f Test accuracy: %.1f%%" % (alpha_reg, accuracy_value))
      accuracy_vals.append(accuracy_value)

Initialized
Alpha val:1.00000 Test accuracy: 59.9%
Initialized
Alpha val:0.61585 Test accuracy: 67.8%
Initialized
Alpha val:0.37927 Test accuracy: 75.8%
Initialized
Alpha val:0.23357 Test accuracy: 80.4%
Initialized
Alpha val:0.14384 Test accuracy: 83.0%
Initialized
Alpha val:0.08859 Test accuracy: 84.7%
Initialized
Alpha val:0.05456 Test accuracy: 85.8%
Initialized
Alpha val:0.03360 Test accuracy: 87.1%
Initialized
Alpha val:0.02069 Test accuracy: 87.3%
Initialized
Alpha val:0.01274 Test accuracy: 87.5%
Initialized
Alpha val:0.00785 Test accuracy: 87.8%
Initialized
Alpha val:0.00483 Test accuracy: 88.0%
Initialized
Alpha val:0.00298 Test accuracy: 87.7%
Initialized
Alpha val:0.00183 Test accuracy: 87.8%
Initialized
Alpha val:0.00113 Test accuracy: 87.8%
Initialized
Alpha val:0.00070 Test accuracy: 87.0%
Initialized
Alpha val:0.00043 Test accuracy: 85.5%
Initialized
Alpha val:0.00026 Test accuracy: 85.2%
Initialized
Alpha val:0.00016 Test accuracy: 85.6%
Initialized
Alpha val:0.00010 T

In [59]:
print("Alpha val:%.5f Test accuracy: %.1f%%" % (alpha_vals[np.argmax(accuracy_vals)], max(accuracy_vals)))

Alpha val:0.00483 Test accuracy: 88.0%


<p>Much better than SGD without regularization.</p>
<p>SGD without regularization: Test accuracy-83.7% </p>
    

In [60]:
# Code taken from 2_fullyconnected and modified by LDP
num_nodes = 1024
batch_size =128
alpha_vals = np.logspace(0,-4,20)
accuracy_vals = []

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_alpha_reg = tf.placeholder(tf.float32)
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))    
    
    # Training computation.
    # Same as above
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_2)) + \
           tf_alpha_reg * tf.nn.l2_loss(weights_1) + \
           tf_alpha_reg * tf.nn.l2_loss(weights_2) 
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training data
    train_prediction = tf.nn.softmax(logits_2)
    
    #Predictions for the validation data
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    #Predictions for the test data
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_2)

In [61]:
num_steps = 3001

for alpha_reg in alpha_vals:
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        # Need to reshape, because the new PlaceHolder has different dimensions
        batch_labels = batch_labels.reshape(batch_size,10)
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_alpha_reg : alpha_reg}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
#         if ((step % 500) == 0):
#           print("Minibatch loss at step %d: %f" % (step, l))
#           print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
#           print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
      accuracy_value = accuracy(test_prediction.eval(), test_labels)
      print("Alpha val:%.5f Test accuracy: %.1f%%" % (alpha_reg, accuracy_value))
      accuracy_vals.append(accuracy_value)

Initialized
Alpha val:1.00000 Test accuracy: 10.0%
Initialized
Alpha val:0.61585 Test accuracy: 29.2%
Initialized
Alpha val:0.37927 Test accuracy: 49.3%
Initialized
Alpha val:0.23357 Test accuracy: 67.8%
Initialized
Alpha val:0.14384 Test accuracy: 80.1%
Initialized
Alpha val:0.08859 Test accuracy: 85.3%
Initialized
Alpha val:0.05456 Test accuracy: 87.3%
Initialized
Alpha val:0.03360 Test accuracy: 88.1%
Initialized
Alpha val:0.02069 Test accuracy: 88.7%
Initialized
Alpha val:0.01274 Test accuracy: 89.5%
Initialized
Alpha val:0.00785 Test accuracy: 90.6%
Initialized
Alpha val:0.00483 Test accuracy: 91.0%
Initialized
Alpha val:0.00298 Test accuracy: 91.8%
Initialized
Alpha val:0.00183 Test accuracy: 91.6%
Initialized
Alpha val:0.00113 Test accuracy: 90.7%
Initialized
Alpha val:0.00070 Test accuracy: 89.2%
Initialized
Alpha val:0.00043 Test accuracy: 89.9%
Initialized
Alpha val:0.00026 Test accuracy: 89.1%
Initialized
Alpha val:0.00016 Test accuracy: 89.4%
Initialized
Alpha val:0.00010 T

In [62]:
print("Alpha val:%.5f Test accuracy: %.1f%%" % (alpha_vals[np.argmax(accuracy_vals)], max(accuracy_vals)))

Alpha val:0.00298 Test accuracy: 91.8%


<p>SGDnn without regularization:89.4%</p>
<p>SGDnn with regularization: much better :) </p> 
<p>The last one perfoms better than the first one. </p>

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [63]:
# Code taken from 2_fullyconnected and modified by LDP
num_nodes = 1024
alpha_reg = alpha_vals[np.argmax(accuracy_vals)]

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))    
    
    # Training computation.
    # Same as above
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_2)) + \
           alpha_reg * tf.nn.l2_loss(weights_1) + \
           alpha_reg * tf.nn.l2_loss(weights_2) 
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training data
    train_prediction = tf.nn.softmax(logits_2)
    
    #Predictions for the validation data
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    #Predictions for the test data
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_2)

In [66]:
num_steps = 3001
batch_size = 128

few_batch_size = 10 * batch_size
small_train_dataset = train_dataset[:few_batch_size, :]
small_train_labels = train_labels[:few_batch_size, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    # Need to reshape, because the new PlaceHolder has different dimensions
    batch_labels = batch_labels.reshape(batch_size,10)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if ((step % 500) == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
      print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1268.086914
Minibatch accuracy: 10.9%
Validation accuracy: 20.0%
Minibatch loss at step 500: 211.047150
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 47.584270
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 10.807323
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 2.581139
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.743086
Minibatch accuracy: 100.0%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.339130
Minibatch accuracy: 99.2%
Validation accuracy: 81.3%
Test accuracy: 87.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [68]:
# Code taken from 2_fullyconnected and modified by LDP
num_nodes = 1024
alpha_reg = alpha_vals[np.argmax(accuracy_vals)]

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))    
    
    # Training computation.
    # Same as above
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    relu_layer = tf.nn.dropout(relu_layer, 0.5)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_2)) + \
           alpha_reg * tf.nn.l2_loss(weights_1) + \
           alpha_reg * tf.nn.l2_loss(weights_2) 
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training data
    train_prediction = tf.nn.softmax(logits_2)
    
    #Predictions for the validation data
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    #Predictions for the test data
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_2)

In [69]:
num_steps = 3001
batch_size = 128


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    # Need to reshape, because the new PlaceHolder has different dimensions
    batch_labels = batch_labels.reshape(batch_size,10)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if ((step % 500) == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
      print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1427.069092
Minibatch accuracy: 11.7%
Validation accuracy: 26.5%
Minibatch loss at step 500: 214.026794
Minibatch accuracy: 78.9%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 47.709648
Minibatch accuracy: 96.9%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 11.151330
Minibatch accuracy: 95.3%
Validation accuracy: 84.7%
Minibatch loss at step 2000: 2.954897
Minibatch accuracy: 93.8%
Validation accuracy: 85.3%
Minibatch loss at step 2500: 1.154325
Minibatch accuracy: 89.8%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 0.696094
Minibatch accuracy: 93.0%
Validation accuracy: 84.7%
Test accuracy: 92.0%


In [70]:
num_steps = 3001
batch_size = 128

few_batch_size = 10 * batch_size
small_train_dataset = train_dataset[:few_batch_size, :]
small_train_labels = train_labels[:few_batch_size, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    # Need to reshape, because the new PlaceHolder has different dimensions
    batch_labels = batch_labels.reshape(batch_size,10)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if ((step % 500) == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
      print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1387.984985
Minibatch accuracy: 14.1%
Validation accuracy: 24.9%
Minibatch loss at step 500: 212.896317
Minibatch accuracy: 99.2%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 47.910011
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 10.857042
Minibatch accuracy: 100.0%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 2.614689
Minibatch accuracy: 100.0%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 0.778454
Minibatch accuracy: 100.0%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.374987
Minibatch accuracy: 99.2%
Validation accuracy: 81.4%
Test accuracy: 87.8%


<p>From results, it doesn't affect the case of overfitting. </p>

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
num_steps = 3001
batch_size = 128


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    # Need to reshape, because the new PlaceHolder has different dimensions
    batch_labels = batch_labels.reshape(batch_size,10)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if ((step % 500) == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
      print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))